<a href="https://colab.research.google.com/github/pedrorostagno/tesis/blob/main/Tesis_nuevo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Usar A100, es el que mejor resultados me dio hasta ahora

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import os
from PIL import Image
import numpy as np
import pickle
from datetime import datetime

dataset_path = '/content/drive/Othercomputers/My Mac/Data/train'

# 🗂️ Carpetas base en Drive
BASE_DIR = "/content/drive/MyDrive/Tesis UTDT"
CACHE_DIR = os.path.join(BASE_DIR, "cache")
EXPERIMENTS_DIR = os.path.join(BASE_DIR, "experimentos")

# ⏱️ Nombre único del experimento basado en fecha y hora
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
RUN_DIR = os.path.join(EXPERIMENTS_DIR, timestamp)

# 📁 Crear carpetas si no existen
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(RUN_DIR, exist_ok=True)

print(f"📁 Cachés en:     {CACHE_DIR}")
print(f"📁 Resultados en: {RUN_DIR}")


📁 Cachés en:     /content/drive/MyDrive/Tesis UTDT/cache
📁 Resultados en: /content/drive/MyDrive/Tesis UTDT/experimentos/20250622-221825


In [62]:
# # 📦 Clase personalizada para cargar el dataset CelebA-Spoof (u otro similar estructurado por carpetas)

# class CelebASpoofDataset(Dataset):
#     def __init__(self, root_dir, transform=None, cache_file=None, max_samples=None):
#         """
#         Inicializa el dataset.

#         Parámetros:
#         - root_dir (str): Ruta raíz del dataset con estructura jerárquica (sujetos > spoof/live > imágenes).
#         - transform (callable, opcional): Transformaciones a aplicar a cada imagen.
#         - cache_file (str, opcional): Ruta para guardar o cargar el cache de rutas e índices.
#         - max_samples (int, opcional): Límite máximo de muestras a cargar (útil para pruebas rápidas).
#         """
#         self.root_dir = root_dir
#         self.transform = transform
#         self.image_paths = []
#         self.labels = []

#         if cache_file is not None and os.path.exists(cache_file):
#             print("Cargando dataset desde cache...")
#             with open(cache_file, 'rb') as f:
#                 self.image_paths, self.labels = pickle.load(f)
#         else:
#             print("Generando la lista de imágenes usando os.scandir...")
#             # Variable para determinar si ya se alcanzó el máximo de muestras
#             max_reached = False
#             with os.scandir(root_dir) as subjects:
#                 for subject in subjects:
#                     if max_samples is not None and len(self.image_paths) >= max_samples:
#                         max_reached = True
#                         break
#                     if subject.is_dir():
#                         subject_path = subject.path
#                         with os.scandir(subject_path) as type_entries:
#                             for entry in type_entries:
#                                 if max_samples is not None and len(self.image_paths) >= max_samples:
#                                     max_reached = True
#                                     break
#                                 if entry.is_dir():
#                                     image_type = entry.name  # 'live' o 'spoof'
#                                     with os.scandir(entry.path) as files:
#                                         for file_entry in files:
#                                             if file_entry.is_file() and file_entry.name.lower().endswith(('.jpg', '.png')):
#                                                 self.image_paths.append(file_entry.path)
#                                                 self.labels.append(0 if image_type == 'spoof' else 1)
#                                                 if max_samples is not None and len(self.image_paths) >= max_samples:
#                                                     max_reached = True
#                                                     break
#                                         if max_samples is not None and len(self.image_paths) >= max_samples:
#                                             break
#                             if max_samples is not None and len(self.image_paths) >= max_samples:
#                                 break
#                     if max_reached:
#                         break
#             # Guardar en cache si se especifica
#             if cache_file is not None:
#                 with open(cache_file, 'wb') as f:
#                     pickle.dump((self.image_paths, self.labels), f)
#                 print("Cache guardado en", cache_file)

#     def __len__(self):
#         """Devuelve la cantidad total de muestras en el dataset."""
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         """
#         Devuelve una imagen y su etiqueta correspondiente.

#         - Abre la imagen en formato RGB.
#         - Aplica transformaciones si fueron especificadas.
#         - Devuelve un tensor y un entero (0=spoof, 1=live).
#         """
#         image_path = self.image_paths[idx]
#         image = Image.open(image_path).convert('RGB')
#         label = self.labels[idx]
#         if self.transform:
#             image = self.transform(image)
#         return image, label

class CelebASpoofDataset(Dataset):
    def __init__(self, root_dir, transform=None, cache_file=None, max_samples=None, balance=False):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        if cache_file is not None and os.path.exists(cache_file):
            print("Cargando dataset desde cache...")
            with open(cache_file, 'rb') as f:
                self.image_paths, self.labels = pickle.load(f)
        else:
            print("Generando la lista de imágenes usando os.scandir...")
            max_reached = False
            with os.scandir(root_dir) as subjects:
                for subject in subjects:
                    if max_samples is not None and len(self.image_paths) >= max_samples:
                        max_reached = True
                        break
                    if subject.is_dir():
                        subject_path = subject.path
                        with os.scandir(subject_path) as type_entries:
                            for entry in type_entries:
                                if max_samples is not None and len(self.image_paths) >= max_samples:
                                    max_reached = True
                                    break
                                if entry.is_dir():
                                    image_type = entry.name
                                    with os.scandir(entry.path) as files:
                                        for file_entry in files:
                                            if file_entry.is_file() and file_entry.name.lower().endswith(('.jpg', '.png')):
                                                self.image_paths.append(file_entry.path)
                                                self.labels.append(0 if image_type == 'spoof' else 1)
                                                if max_samples is not None and len(self.image_paths) >= max_samples:
                                                    max_reached = True
                                                    break
                                        if max_reached:
                                            break
                            if max_reached:
                                break
                    if max_reached:
                        break
            if cache_file is not None:
                with open(cache_file, 'wb') as f:
                    pickle.dump((self.image_paths, self.labels), f)
                print("Cache guardado en", cache_file)

        # ⚖️ Balanceo posterior (solo si balance=True)
        if balance:
            print("⚖️ Aplicando balance de clases...")
            spoof_samples = [(p, l) for p, l in zip(self.image_paths, self.labels) if l == 0]
            live_samples  = [(p, l) for p, l in zip(self.image_paths, self.labels) if l == 1]
            min_len = min(len(spoof_samples), len(live_samples))

            # Cortamos la clase mayoritaria
            spoof_samples = spoof_samples[:min_len]
            live_samples = live_samples[:min_len]

            combined = spoof_samples + live_samples
            random.shuffle(combined)
            self.image_paths, self.labels = zip(*combined)
            self.image_paths = list(self.image_paths)
            self.labels = list(self.labels)

            print(f"📊 Dataset balanceado: {min_len} spoof + {min_len} live = {2 * min_len} total")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label




In [63]:

# 📁 Definición de rutas a cada partición del dataset (entrenamiento, validación y test)
train_data_dir = '/content/drive/Othercomputers/My Mac/Data/train/train'
val_data_dir   = '/content/drive/Othercomputers/My Mac/Data/train/val'
test_data_dir  = '/content/drive/Othercomputers/My Mac/Data/train/test'

# 📐 Dimensiones estándar a las que se redimensionarán todas las imágenes
img_width, img_height = 224, 224

# ⚙️ Tamaño del batch para los DataLoaders
batch_size = 128

# 🌀 Transformaciones de preprocesamiento para cada conjunto de datos
data_transforms = {
    # 'train': transforms.Compose([
    #     transforms.Resize((img_width, img_height)),              # Redimensiona la imagen a 224x224
    #     transforms.RandomHorizontalFlip(),                       # Aplica flip horizontal aleatorio (augmentación)
    #     transforms.ToTensor(),                                   # Convierte PIL Image a tensor
    #     transforms.Normalize([0.485, 0.456, 0.406],              # Normalización con media y desvío estándar de ImageNet
    #                          [0.229, 0.224, 0.225])
    # ]),
    'train': transforms.Compose([
        transforms.Resize((img_width + 32, img_height + 32)),    # Aumenta resolución previa a crop
        transforms.RandomResizedCrop((img_width, img_height), scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.02),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),
        transforms.RandomApply([transforms.RandomRotation(10)], p=0.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


<details>

¿Por qué se redimensiona a (224, 224)?

Motivo principal: muchos modelos preentrenados (como MobileNetV2, ResNet, VGG) fueron entrenados originalmente con imágenes de 224x224 píxeles.
Usar ese tamaño:
Permite reutilizar pesos preentrenados (transfer learning) sin modificar la arquitectura.
Asegura una entrada consistente para la red (todas las imágenes con el mismo tamaño).
Además, 224×224 es un buen balance entre:
Calidad visual suficiente para tareas de clasificación.
Eficiencia computacional (no tan pesado como 512x512, por ejemplo).



¿Qué significa batch_size = 32 y por qué ese número?

El batch size es la cantidad de imágenes que se procesan simultáneamente en una pasada (forward + backward) durante el entrenamiento.
Tamaño común: 8, 16, 32, 64, etc.
Usar 32 implica:
Buena estabilidad numérica del gradiente.
Razonable uso de memoria en GPU (ni muy chico ni muy grande).
En general:
Batches chicos (8-16): más precisos pero lentos.
Batches grandes (64-128+): más rápidos pero requieren más memoria y pueden converger peor.
Batch = 32 es una elección segura, ampliamente utilizada, y probablemente compatible con el uso de GPU en Colab.



Normalizacion

Se utiliza la normalización estándar de ImageNet (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) porque el modelo base empleado fue preentrenado sobre este dataset. Esta normalización asegura que las imágenes de entrada tengan una distribución similar a la vista durante el preentrenamiento, lo cual es esencial para transfer learning efectivo.

</details>

In [67]:
# Cantidad máxima de muestras por partición (ajustable)
train_max_samples = 50000
val_max_samples = 10000
test_max_samples = 5000

# 📦 Dataset de entrenamiento con cache y transformación
train_dataset = CelebASpoofDataset(
    root_dir=train_data_dir,
    transform=data_transforms['train'],
    cache_file=os.path.join(CACHE_DIR, "train_dataset_cache_1.pkl"),
    max_samples=train_max_samples,
    balance=True
    # max_subjects=300
)

# 📦 Dataset de validación
val_dataset = CelebASpoofDataset(
    root_dir=val_data_dir,
    transform=data_transforms['val'],
    cache_file=os.path.join(CACHE_DIR, "val_dataset_cache_1.pkl"),
    max_samples=val_max_samples,
    # balance=True
    # max_subjects=300
)

# 📦 Dataset de test
# test_dataset = CelebASpoofDataset(
#     root_dir=test_data_dir,
#     transform=data_transforms['test'],
#     cache_file=os.path.join(CACHE_DIR, "test_dataset_cache.pkl"),
#     max_samples=test_max_samples
#     # max_subjects=300
# )

# 🔄 DataLoaders para alimentar los datos al modelo en mini-batches
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,     # Cantidad de muestras por batch (definido previamente como 32)
    shuffle=True,              # Mezcla los datos en cada época (importante para entrenamiento)
    num_workers=8,             # Procesos paralelos para cargar datos (ajustar según GPU/Colab)
    pin_memory=True            # Optimiza transferencia a GPU (si se usa CUDA)
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,             # No se mezcla para evaluación consistente
    num_workers=8,
    pin_memory=True
)

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=batch_size,
#     shuffle=False,
#     num_workers=8,
#     pin_memory=True
# )


Cargando dataset desde cache...
⚖️ Aplicando balance de clases...
📊 Dataset balanceado: 17123 spoof + 17123 live = 34246 total
Generando la lista de imágenes usando os.scandir...
Cache guardado en /content/drive/MyDrive/Tesis UTDT/cache/val_dataset_cache_1.pkl


In [68]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 🧠 Carga de modelo preentrenado
model = models.mobilenet_v2(pretrained=True)

# 🔧 Modificación de la última capa para clasificación binaria
# Extraemos el número de entradas de la capa final
num_ftrs = model.classifier[1].in_features

# Reemplazamos la última capa por una nueva totalmente conectada con 2 salidas (spoof / live)
model.classifier[1] = nn.Linear(num_ftrs, 2)


# ⚙️ Optimizador
# Adam es un optimizador eficiente y ampliamente utilizado
optimizer = optim.Adam(model.parameters(), lr=0.0005) #Cambie el learning rate de 0.001 a 0.0005
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<details>
MobileNetV2
Ventajas principales:
Modelo ligero y eficiente: MobileNetV2 está diseñado para ser rápido y liviano, ideal si usás Colab, entornos con GPU limitada o pensás implementarlo en dispositivos móviles o edge.
Alto rendimiento: A pesar de ser liviano, alcanza buena precisión en tareas de clasificación.
Transfer learning efectivo: Funciona muy bien con fine-tuning para tareas con pocos datos (como en muchos proyectos de tesis).
✅ Justificación para tu tesis:
MobileNetV2 se eligió por su eficiencia computacional y alto rendimiento en clasificación de imágenes, siendo especialmente útil en contextos con recursos limitados o para aplicaciones en tiempo real. Además, su arquitectura permite realizar transfer learning de forma efectiva.



📥 ¿Por qué pretrained=True?
Significado:
Carga pesos ya entrenados en el dataset ImageNet (más de 1M de imágenes en 1000 clases).
Esto significa que las primeras capas del modelo ya están optimizadas para detectar características visuales genéricas como bordes, texturas, patrones, etc.
Ventajas:
Mejora la convergencia (entrena más rápido).
Mejora la precisión, especialmente si tenés poco volumen de datos.
Es la base del Transfer Learning.
Justificación para tu tesis:
Se utilizan pesos preentrenados sobre ImageNet para aprovechar características visuales genéricas aprendidas previamente, acelerando el entrenamiento y mejorando el rendimiento con un volumen de datos moderado.


¿Por qué CrossEntropyLoss?
¿Qué hace?
Es la función de pérdida estándar para clasificación multi-clase (incluye binaria).
Combina LogSoftmax + Negative Log Likelihood, calculando la discrepancia entre las probabilidades predichas y la clase verdadera.
Justificación:
Es adecuada cuando las etiquetas están codificadas como enteros (0 o 1).
Automáticamente maneja logits (sin necesidad de aplicar softmax manual).
Justificación para tu tesis:
Se utiliza la función de pérdida CrossEntropyLoss, ampliamente aceptada para problemas de clasificación, ya que penaliza las predicciones incorrectas proporcionalmente a su confianza, incentivando al modelo a asignar probabilidades más altas a las clases correctas.



⚙️ ¿Por qué Adam y lr=0.001?

¿Qué es Adam?
Un optimizador adaptativo que ajusta la tasa de aprendizaje individualmente para cada peso.
Combina ventajas de:
RMSProp (ajuste por histórico de gradientes)
Momentum (aceleración del aprendizaje)
¿Por qué lr = 0.001?
Es el valor por defecto y recomendado para Adam.
Proporciona un balance entre estabilidad y velocidad en la mayoría de los casos.
Si lo aumentás (por ej. 0.01):
El modelo puede aprender más rápido pero también volverse inestable.
Si lo bajás (por ej. 0.0001):
El modelo aprende más lento, pero más establemente (útil si tenés overfitting o datos ruidosos).
Justificación para tu tesis:
Se utiliza el optimizador Adam por su capacidad adaptativa y eficiencia, permitiendo un entrenamiento más rápido y estable sin necesidad de ajustes manuales constantes. La tasa de aprendizaje inicial (lr = 0.001) es un valor estándar que ofrece un equilibrio entre velocidad de convergencia y estabilidad numérica.
</details>


In [69]:
# ⚙️ Detectar el dispositivo: GPU si está disponible, si no usa CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 🧠 Mover el modelo al dispositivo seleccionado
model = model.to(device)

# 📊 Calcular la distribución de clases en el dataset de entrenamiento
labels = np.array(train_dataset.labels)
unique, counts = np.unique(labels, return_counts=True)
print(f"Distribución de clases en entrenamiento: {dict(zip(unique, counts))}")

# 🧮 Calcular pesos inversamente proporcionales a la cantidad de muestras por clase
total = counts.sum()
weights = [total / c for c in counts]  # Más peso a la clase menos representada

# Convertir a tensor y mover a GPU (si está disponible)
weights_tensor = torch.FloatTensor(weights).to(device)
print("Pesos de clases:", weights_tensor)

# 🎯 Definir una función de pérdida ponderada para tratar el desbalance de clases
criterion = nn.CrossEntropyLoss(weight=weights_tensor)


Distribución de clases en entrenamiento: {np.int64(0): np.int64(17123), np.int64(1): np.int64(17123)}
Pesos de clases: tensor([2., 2.], device='cuda:0')


<details>
El dataset de entrenamiento puede estar desbalanceado, es decir, tener más ejemplos de una clase que de otra. Para evitar que el modelo aprenda a favorecer la clase mayoritaria, se calcula la distribución real de clases y se asignan pesos inversamente proporcionales a su frecuencia. Estos pesos se utilizan en la función CrossEntropyLoss para penalizar más fuertemente los errores en las clases menos representadas. Esto mejora la equidad del modelo y su capacidad de detectar ambas clases por igual.


criterion = nn.CrossEntropyLoss(weight=weights_tensor)
Con pesos según distribución
Aplica mayor penalización a la clase minoritaria.
Específicamente útil si tu problema tiene desbalance de clases.
Mejora métricas como recall, F1-score macro y balanced accuracy.
</details>

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.tensorboard import SummaryWriter
import sklearn.metrics as metrics
import pandas as pd
import json

# 🔁 Función para entrenar una sola época usando Mixed Precision
def train_epoch(model, train_loader, optimizer, criterion, device, scaler):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # ⚙️ Entrenamiento en precisión mixta (float16 + float32)
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples
    return epoch_loss, epoch_acc

# 📊 Función para evaluar el modelo en validación y obtener métricas
def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    total_samples = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = val_loss / total_samples
    epoch_acc = val_corrects.double() / total_samples
    report = metrics.classification_report(all_labels, all_preds, output_dict=True, zero_division=0)

    return epoch_loss, epoch_acc, report, all_preds, all_labels


import pandas as pd


# 🚀 Función principal de entrenamiento con validación, checkpoints, early stopping y logging
def train_model(model, train_loader, val_loader, criterion, optimizer, device,
                num_epochs=10, patience=3, checkpoint_interval=1, log_dir=None):

    if log_dir is None:
        log_dir = os.path.join(RUN_DIR, "tensorboard")
    os.makedirs(log_dir, exist_ok=True)
    writer = SummaryWriter(log_dir=log_dir)

    # 📝 Guardar configuración
    config = {
        "num_epochs": num_epochs,
        "patience": patience,
        "checkpoint_interval": checkpoint_interval,
        "batch_size": train_loader.batch_size,
        "optimizer": type(optimizer).__name__,
        "learning_rate": optimizer.param_groups[0]["lr"],
    }
    with open(os.path.join(RUN_DIR, "config.json"), "w") as f:
        json.dump(config, f, indent=4)

    scaler = GradScaler()
    best_val_loss = float('inf')
    trigger_times = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}\n' + '-' * 10)

        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device, scaler)
        val_loss, val_acc, report, all_preds, all_labels = validate_epoch(model, val_loader, criterion, device)

        # para modificar el learning rate dinamicamente
        scheduler.step(val_loss)

        # Mostrar el LR actual
        for i, param_group in enumerate(optimizer.param_groups):
            print(f"🔁 Learning Rate actual: {scheduler.get_last_lr()[0]:.6f}")

        print(f'Train Loss: {train_loss:.4f}  Acc: {train_acc:.4f}')
        print(f'Val Loss:   {val_loss:.4f}  Acc: {val_acc:.4f}')
        print(metrics.classification_report(all_labels, all_preds, target_names=["spoof", "live"], zero_division=0))

        # TensorBoard logs
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Acc/train', train_acc, epoch)
        writer.add_scalar('Acc/val', val_acc, epoch)
        writer.add_scalar('F1/macro_val', report['macro avg']['f1-score'], epoch)

        # Early stopping + mejor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trigger_times = 0
            best_model_path = os.path.join(RUN_DIR, "best_model.pth")
            torch.save(model.state_dict(), best_model_path)
            print(f'✅ Best model saved to {best_model_path}')
        else:
            trigger_times += 1
            print(f'Early stopping trigger: {trigger_times}/{patience}')
            if trigger_times >= patience:
                print("⛔ Early stopping!")
                break

        # Checkpoint por época
        if (epoch + 1) % checkpoint_interval == 0:
            checkpoint_path = os.path.join(RUN_DIR, f'checkpoint_epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'train_acc': train_acc.item(),
                'val_loss': val_loss,
                'val_acc': val_acc.item(),
            }, checkpoint_path)
            print(f'📦 Checkpoint saved to {checkpoint_path}')

    writer.close()
    return model, train_loss, train_acc, val_loss, val_acc, report



<details>
<summary><strong>📚 Explicación detallada de funciones de entrenamiento</strong></summary>

### 🔁 `train_epoch(...)`
Esta función entrena el modelo durante una sola época usando *Mixed Precision Training* para mejorar el rendimiento y reducir el consumo de memoria en GPU.

**Parámetros:**
- `model`: modelo de red neuronal.
- `train_loader`: `DataLoader` con los datos de entrenamiento.
- `optimizer`: optimizador (por ejemplo, Adam).
- `criterion`: función de pérdida.
- `device`: CPU o GPU.
- `scaler`: `GradScaler` para entrenamiento en precisión mixta.

**Qué hace:**
- Activa el modo entrenamiento del modelo.
- Para cada batch:
  - Ejecuta forward pass (predicción).
  - Calcula la pérdida.
  - Ejecuta backward pass y actualiza pesos (gradientes escalados).
  - Acumula métricas: pérdida total y aciertos.

**Devuelve:**
- `epoch_loss`: pérdida media de la época.
- `epoch_acc`: exactitud (accuracy) de entrenamiento.

---

### 📊 `validate_epoch(...)`
Evalúa el modelo en el conjunto de validación y calcula métricas detalladas.

**Parámetros:**
- `model`: modelo entrenado.
- `val_loader`: datos de validación.
- `criterion`: función de pérdida.
- `device`: CPU o GPU.

**Qué hace:**
- Activa el modo evaluación del modelo.
- Desactiva cálculo de gradientes.
- Calcula pérdida, predicciones y métricas en todo el conjunto.
- Genera un `classification_report` con precisión, recall y F1.

**Devuelve:**
- `epoch_loss`: pérdida media.
- `epoch_acc`: exactitud (accuracy).
- `report`: diccionario con métricas detalladas (`f1-score`, `precision`, etc.).
- `all_preds`: predicciones crudas.
- `all_labels`: etiquetas verdaderas.

---

### 🚀 `train_model(...)`
Función principal de entrenamiento que combina las anteriores. Agrega:
- Registro en TensorBoard.
- Early Stopping.
- Guardado de checkpoints y del mejor modelo.

**Parámetros:**
- `model`: red neuronal.
- `train_loader`, `val_loader`: `DataLoader`s para entrenamiento y validación.
- `criterion`: función de pérdida.
- `optimizer`: algoritmo de optimización.
- `device`: CPU o GPU.
- `num_epochs`: total de épocas.
- `patience`: épocas sin mejora antes de detener.
- `checkpoint_interval`: cada cuántas épocas guardar.
- `log_dir`: carpeta de logs de TensorBoard.

**Qué hace:**
- Itera por época:
  - Entrena una época (`train_epoch`)
  - Valida la época (`validate_epoch`)
  - Registra métricas en TensorBoard
  - Guarda el mejor modelo
  - Aplica Early Stopping si no mejora
  - Guarda checkpoints periódicos

**Devuelve:**
- El modelo final entrenado.

</details>


In [71]:
# continua del bloque anterior, pero lo pongo aca para tenerlo separado
# 🧪 Ejecución del entrenamiento (se asume que todo está definido)
trained_model, train_loss, train_acc, val_loss, val_acc, val_report = train_model(
    model, train_loader, val_loader,
    criterion, optimizer, device,
    num_epochs=10, patience=3
)

Epoch 1/10
----------


/tmp/ipython-input-70-2456198354.py:95: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-70-2456198354.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipython-input-70-2456198354.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔁 Learning Rate actual: 0.000500
Train Loss: 0.0388  Acc: 0.9856
Val Loss:   0.0673  Acc: 0.9756
              precision    recall  f1-score   support

       spoof       1.00      0.96      0.98      6507
        live       0.94      1.00      0.97      3493

    accuracy                           0.98     10000
   macro avg       0.97      0.98      0.97     10000
weighted avg       0.98      0.98      0.98     10000

✅ Best model saved to /content/drive/MyDrive/Tesis UTDT/experimentos/20250622-221825/best_model.pth
📦 Checkpoint saved to /content/drive/MyDrive/Tesis UTDT/experimentos/20250622-221825/checkpoint_epoch_1.pth
Epoch 2/10
----------


/tmp/ipython-input-70-2456198354.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipython-input-70-2456198354.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔁 Learning Rate actual: 0.000500
Train Loss: 0.0140  Acc: 0.9952
Val Loss:   0.0723  Acc: 0.9760
              precision    recall  f1-score   support

       spoof       1.00      0.96      0.98      6507
        live       0.94      1.00      0.97      3493

    accuracy                           0.98     10000
   macro avg       0.97      0.98      0.97     10000
weighted avg       0.98      0.98      0.98     10000

Early stopping trigger: 1/3
📦 Checkpoint saved to /content/drive/MyDrive/Tesis UTDT/experimentos/20250622-221825/checkpoint_epoch_2.pth
Epoch 3/10
----------


/tmp/ipython-input-70-2456198354.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipython-input-70-2456198354.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔁 Learning Rate actual: 0.000250
Train Loss: 0.0122  Acc: 0.9957
Val Loss:   0.0695  Acc: 0.9761
              precision    recall  f1-score   support

       spoof       1.00      0.96      0.98      6507
        live       0.94      1.00      0.97      3493

    accuracy                           0.98     10000
   macro avg       0.97      0.98      0.97     10000
weighted avg       0.98      0.98      0.98     10000

Early stopping trigger: 2/3
📦 Checkpoint saved to /content/drive/MyDrive/Tesis UTDT/experimentos/20250622-221825/checkpoint_epoch_3.pth
Epoch 4/10
----------


/tmp/ipython-input-70-2456198354.py:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipython-input-70-2456198354.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔁 Learning Rate actual: 0.000250
Train Loss: 0.0058  Acc: 0.9983
Val Loss:   0.0805  Acc: 0.9734
              precision    recall  f1-score   support

       spoof       1.00      0.96      0.98      6507
        live       0.93      1.00      0.96      3493

    accuracy                           0.97     10000
   macro avg       0.96      0.98      0.97     10000
weighted avg       0.98      0.97      0.97     10000

Early stopping trigger: 3/3
⛔ Early stopping!


In [72]:
import numpy as np
import sklearn.metrics as metrics
import seaborn as sns
import matplotlib.pyplot as plt
import torch

# 🧠 Cargar el mejor modelo desde el directorio de la corrida actual
best_model = models.mobilenet_v2(pretrained=False)
num_ftrs = best_model.classifier[1].in_features
best_model.classifier[1] = nn.Linear(num_ftrs, 2)
best_model.load_state_dict(torch.load(os.path.join(RUN_DIR, 'best_model.pth')))
best_model = best_model.to(device)
print("✅ Mejor modelo cargado desde RUN_DIR")



def test_model(model, test_loader, criterion, device, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model.eval()
    all_preds = []
    all_labels = []
    test_loss = 0.0
    total_samples = 0

    # Usar autocast para precisión mixta en la inferencia
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            test_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = test_loss / total_samples
    print(f'Test Loss: {avg_loss:.4f}')

    # Convertir a arrays de numpy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Calcular matriz de confusión y reporte en formato dict
    cm = metrics.confusion_matrix(all_labels, all_preds)
    report = metrics.classification_report(all_labels, all_preds, target_names=["spoof", "live"], zero_division=0)
    report_dict = metrics.classification_report(all_labels, all_preds, target_names=["spoof", "live"], output_dict=True, zero_division=0)

    # Guardar métricas como JSON
    with open(os.path.join(output_dir, 'classification_report.json'), 'w') as f:
        json.dump(report, f, indent=4)
    print("📄 Reporte de clasificación guardado en JSON.")

    print("Matriz de Confusión:")
    print(cm)
    print("Reporte de Clasificación:")
    print(report)

    # Guardar matriz de confusión como imagen PNG
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["spoof", "live"], yticklabels=["spoof", "live"])
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.title("Matriz de Confusión")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    plt.close()
    print("🖼️ Matriz de confusión guardada como imagen.")

    return avg_loss, report_dict, cm




✅ Mejor modelo cargado desde RUN_DIR


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<details>
<summary><strong>🧪 Evaluación del modelo sobre el conjunto de test</strong></summary>

Esta etapa evalúa el modelo entrenado sobre un conjunto independiente de prueba. La función `test_model` se encarga de calcular:

- La pérdida promedio (`test_loss`)
- Métricas de clasificación como:
  - Accuracy
  - Precision
  - Recall
  - F1-score (por clase y macro)
- La matriz de confusión
- Una visualización gráfica con `seaborn`

### 🔍 Detalles técnicos:
- El modelo se evalúa en modo `eval()` y sin cálculo de gradientes (`no_grad()`).
- Se usa `autocast()` para inferencia eficiente con precisión mixta (float16).
- Las predicciones y etiquetas se acumulan para calcular métricas globales con `sklearn.metrics`.

### 📊 Resultado esperado:
- Impresión por consola del `classification_report`.
- Visualización clara de errores y aciertos con una matriz de confusión coloreada.
- Retorno de:
  - `avg_loss`: pérdida total dividida por cantidad de muestras.
  - `report`: diccionario con todas las métricas.
  - `cm`: matriz de confusión en formato NumPy.

</details>


In [73]:
# Continuacion del bloque anterior. Testeo del modelo en otro dataset
test_data_dir2 = '/content/drive/Othercomputers/My Mac/Data/test'

test_dataset2 = CelebASpoofDataset(
    root_dir=test_data_dir2,
    transform=data_transforms['test'],
    cache_file=os.path.join(CACHE_DIR, 'test_dataset_cache_full.pkl'),
    max_samples=10000
    # max_subjects=300
)


# DataLoaders
test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)


# Ejemplo de uso:
# test_loss, test_report, test_cm = test_model(model, test_loader2, criterion, device)
test_loss, test_report, test_cm = test_model(best_model, test_loader2, criterion, device, output_dir=RUN_DIR)



Cargando dataset desde cache...


/tmp/ipython-input-72-328312363.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Test Loss: 2.5762
📄 Reporte de clasificación guardado en JSON.
Matriz de Confusión:
[[1353 3972]
 [   7 4668]]
Reporte de Clasificación:
              precision    recall  f1-score   support

       spoof       0.99      0.25      0.40      5325
        live       0.54      1.00      0.70      4675

    accuracy                           0.60     10000
   macro avg       0.77      0.63      0.55     10000
weighted avg       0.78      0.60      0.54     10000

🖼️ Matriz de confusión guardada como imagen.


In [74]:
import os
import pandas as pd

def log_experiment_summary(run_dir,
                           train_loss, train_acc,
                           val_loss, val_acc, val_report,
                           test_loss, test_report,
                           train_max_samples, val_max_samples, test_max_samples):
    summary_path = os.path.join(EXPERIMENTS_DIR, "all_runs.csv")
    run_name = os.path.basename(run_dir)

    # Utilidad segura para extraer métricas, aunque una clase no esté presente
    def get_metric(report, class_name, metric):
        return round(report.get(class_name, {}).get(metric, 0.0), 4)

    summary_data = {
        "run_name": run_name,

        # Métricas de entrenamiento
        "train_loss": round(train_loss, 4),
        "train_acc": round(train_acc.item(), 4),

        # Métricas de validación
        "val_loss": round(val_loss, 4),
        "val_acc": round(val_acc.item(), 4),
        "f1_macro_val": get_metric(val_report, "macro avg", "f1-score"),
        "precision_spoof_val": get_metric(val_report, "0", "precision"),
        "recall_spoof_val": get_metric(val_report, "0", "recall"),
        "f1_spoof_val": get_metric(val_report, "0", "f1-score"),
        "precision_live_val": get_metric(val_report, "1", "precision"),
        "recall_live_val": get_metric(val_report, "1", "recall"),
        "f1_live_val": get_metric(val_report, "1", "f1-score"),

        # Métricas de test final
        "test_loss": round(test_loss, 4),
        "f1_macro_test": get_metric(test_report, "macro avg", "f1-score"),
        "precision_spoof_test": get_metric(test_report, "spoof", "precision"),
        "recall_spoof_test": get_metric(test_report, "spoof", "recall"),
        "f1_spoof_test": get_metric(test_report, "spoof", "f1-score"),
        "precision_live_test": get_metric(test_report, "live", "precision"),
        "recall_live_test": get_metric(test_report, "live", "recall"),
        "f1_live_test": get_metric(test_report, "live", "f1-score"),

        # Info de dataset
        "train_max_samples": train_max_samples,
        "val_max_samples": val_max_samples,
        "test_max_samples": test_max_samples
    }

    df = pd.DataFrame([summary_data])
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', header=False, index=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"📝 Resumen de la corrida guardado en {summary_path}")


In [75]:
test_report

{'spoof': {'precision': 0.9948529411764706,
  'recall': 0.2540845070422535,
  'f1-score': 0.40478683620044875,
  'support': 5325.0},
 'live': {'precision': 0.5402777777777777,
  'recall': 0.9985026737967915,
  'f1-score': 0.7011641006383778,
  'support': 4675.0},
 'accuracy': 0.6021,
 'macro avg': {'precision': 0.7675653594771241,
  'recall': 0.6262935904195225,
  'f1-score': 0.5529754684194133,
  'support': 10000.0},
 'weighted avg': {'precision': 0.7823390522875817,
  'recall': 0.6021,
  'f1-score': 0.5433432073251805,
  'support': 10000.0}}

In [76]:
log_experiment_summary(
    run_dir=RUN_DIR,
    train_loss=train_loss,
    train_acc=train_acc,
    val_loss=val_loss,
    val_acc=val_acc,
    val_report=val_report,
    test_loss=test_loss,
    test_report=test_report,
    train_max_samples=train_max_samples,
    val_max_samples=val_max_samples,
    test_max_samples=test_max_samples
)


📝 Resumen de la corrida guardado en /content/drive/MyDrive/Tesis UTDT/experimentos/all_runs.csv


In [47]:
# Mostrar 10 predicciones incorrectas con su path
for i, (img, label) in enumerate(test_dataset):
    pred = model(img.unsqueeze(0).to(device)).argmax(dim=1).item()
    if pred != label:
        print(f"ERROR → Pred: {pred} | Real: {label} | Path: {test_dataset.image_paths[i]}")
        if i > 10: break


ERROR → Pred: 1 | Real: 0 | Path: /content/drive/Othercomputers/My Mac/Data/train/test/9893/spoof/192739.jpg
ERROR → Pred: 1 | Real: 0 | Path: /content/drive/Othercomputers/My Mac/Data/train/test/9893/spoof/179815.jpg
